# Quotes LTP

In [1]:
import pandas as pd
from breeze_connect import BreezeConnect
from datetime import date
from selenium import webdriver
from pyotp import TOTP
import urllib
import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
file = open("security.txt", "r")
keys = file.read().split() 
api_key = keys[0]
key_secret = keys[1]
userID = keys[2]
pwd = keys[3]
totp_key = keys[4]
breeze = BreezeConnect(api_key=api_key)

In [3]:
browser = webdriver.Chrome()
browser.get("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(api_key))
browser.implicitly_wait(5)
username = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[1]/input')
password = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[3]/div/input') 
username.send_keys(userID)
password.send_keys(pwd)
#Checkbox
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[4]/div/input').click()
# Click Login Button
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[1]/div[2]/div/div[5]/input[1]').click()
time.sleep(2)
pin = browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[3]/div/div[1]/input')
totp = TOTP(totp_key)
token = totp.now()
pin.send_keys(token)
browser.find_element("xpath", '/html/body/form/div[2]/div/div/div[2]/div/div[2]/div[2]/div[4]/input[1]').click()
time.sleep(3)
temp_token=browser.current_url.split('apisession=')[1][:8]
print('temp_token', temp_token)

temp_token 6294085


In [4]:
breeze.generate_session(api_secret=key_secret,session_token=temp_token)

### Get Quote

In [5]:
stock_code = 'TCS'
data = breeze.get_quotes(stock_code=stock_code, exchange_code="NSE", product_type="cash")
put_data = pd.DataFrame(data["Success"])
df = pd.DataFrame()
if put_data.empty:
    print('Data not found:')
else:
    df = put_data
df

,exchange_code,product_type,stock_code,expiry_date,right,strike_price,ltp,ltt,best_bid_price,best_bid_quantity,...,best_offer_quantity,open,high,low,previous_close,ltp_percent_change,upper_circuit,lower_circuit,total_quantity_traded,spot_price
0,NSE,,TCS,None,None,0.0,3179.30,17-Mar-2023 15:59:49,0.0,0,...,3323,3150.5,3221.4,3144.0,3185.00,0.178964,3497.2,2861.4,6739966,None
1,BSE,,TCS,None,None,0.0,3178.95,17-Mar-2023 15:31:19,3172.2,1,...,30,3151.0,3220.0,3145.0,3184.75,0.182118,3496.8,2861.1,269544,None


In [6]:
df['ltp'].iloc[0]

3179.3